In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay, roc_curve,auc, precision_recall_curve, average_precision_score, f1_score, auc 
from sklearn.feature_selection import RFECV, RFE
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from imblearn.metrics import geometric_mean_score
import json
import seaborn as sns
import warnings
import pickle
import shap
warnings.filterwarnings("ignore")
sns.set_theme()

from tqdm import tqdm

In [173]:
X_train = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/분석/분석/[24-12-11] rfe feature selection data/X_train.csv')
X_test = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/분석/분석/[24-12-11] rfe feature selection data/X_test.csv')    
y_train = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/분석/분석/[24-12-11] rfe feature selection data/y_train.csv')
y_test = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/분석/분석/[24-12-11] rfe feature selection data/y_test.csv')    


X_train = X_train.drop(X_train.columns[0], axis=1)
X_test = X_test.drop(X_test.columns[0], axis=1)
y_train = y_train.drop(y_train.columns[0], axis=1)
y_test = y_test.drop(y_test.columns[0], axis=1)

In [ ]:
min_features_to_select = 2
step = 1

fit_LR = LogisticRegression()
cv = StratifiedKFold(n_splits=5)

rfecv = RFECV(estimator=fit_LR, min_features_to_select=min_features_to_select, step=step, cv=cv, scoring='accuracy')

rfecv.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(12, 9))
plt.title('Recursive Feature Elimination with 5-fold Stratified cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('Mean test score (accuracy)', fontsize=14, labelpad=20)
plt.plot(range(min_features_to_select, len(rfecv.cv_results_['mean_test_score'])*step + min_features_to_select, step), rfecv.cv_results_['mean_test_score'], color='blue', linewidth=3)
# plt.vlines(x=5, ymin=np.min(rfecv.cv_results_['mean_test_score']), ymax=0.9, color='gray', linestyle='dotted', linewidth=3)

In [ ]:
selector = RFE(fit_LR, n_features_to_select=5)
selector.fit(X_train,y_train)

In [ ]:
X_train.columns[selector.support_]